In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_community.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
key=os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(
    openai_api_key=key,
    model_name="gpt-3.5-turbo",
    temperature=0.5
)

d:\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
response_json={
    "1":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "2":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "3":{
        "mcq":"Multiple Choice Question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
}

In [8]:
template="""
    Text:{text}
    You are an expert MCQ maker. Given the above text, it is your job to \
    Create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Mkae sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
    Ensure to make {number} MCQs
    ### RESPONSE_JSON
    {response_json}
"""

In [9]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template
)

In [10]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

d:\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
template2="""
    You are an expert english grammerian writer. Given a Multiple Choice Quiz for {subject} students.\
    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
    if the quiz is not at per with cognitive and analytical abilities of the students,\
    Upadate the quiz questions which needs to be change the tone such that it perfectly fits the students abilities
    Quiz_MCQs:
    {quiz}

    Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=template2
)

In [13]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [15]:
file_path=r"D:\mcqgen\data.txt"

In [16]:
file_path

'D:\\mcqgen\\data.txt'

In [17]:
with open(file_path,"r") as fp:
    text=fp.read()

In [18]:
print(text)

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once somet

In [19]:
json.dumps(response_json)

'{"1": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "Multiple Choice Question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
number=5
subject="Artificial Intelligence"
tone="Simple"

In [21]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": number,
            "subject":subject,
            "tone": tone,
            "response_json": json.dumps(response_json)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text:Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A

d:\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are an expert english grammerian writer. Given a Multiple Choice Quiz for Artificial Intelligence students.    You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
    if the quiz is not at per with cognitive and analytical abilities of the students,    Upadate the quiz questions which needs to be change the tone such that it perfectly fits the students abilities
    Quiz_MCQs:
    {
    "1": {
        "mcq": "What is the broadest sense of Artificial Intelligence (AI)?",
        "options": {
            "a": "Intelligence exhibited by humans",
            "b": "Intelligence exhibited by machines",
            "c": "Intelligence exhibited by animals",
            "d": "Intelligence exhibited by plants"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Who was the first person to conduct substantial rese

In [22]:
print(f"Total Tokens :{cb.total_tokens}")
print(f"Prompt Tokens :{cb.prompt_tokens}")
print(f"Completion Tokens :{cb.completion_tokens}")
print(f"Total cost :{cb.total_cost}")

Total Tokens :1841
Prompt Tokens :1388
Completion Tokens :453
Total cost :0.0029879999999999998


In [23]:
response

{'text': 'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1] Such machines may be called AIs.\n\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).[2] However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI becaus

In [24]:
quiz=response.get("quiz")

In [25]:
json.loads(quiz)

{'1': {'mcq': 'What is the broadest sense of Artificial Intelligence (AI)?',
  'options': {'a': 'Intelligence exhibited by humans',
   'b': 'Intelligence exhibited by machines',
   'c': 'Intelligence exhibited by animals',
   'd': 'Intelligence exhibited by plants'},
  'correct': 'b'},
 '2': {'mcq': 'Who was the first person to conduct substantial research in machine intelligence?',
  'options': {'a': 'John McCarthy',
   'b': 'Marvin Minksy',
   'c': 'Nathaniel Rochester',
   'd': 'Alan Turing'},
  'correct': 'd'},
 '3': {'mcq': 'What is the field of research in computer science that develops methods and software for machines to perceive their environment?',
  'options': {'a': 'Robotics',
   'b': 'Artificial Intelligence',
   'c': 'Data Science',
   'd': 'Cybersecurity'},
  'correct': 'b'},
 '4': {'mcq': 'Which technique surpassed all previous AI techniques after 2012?',
  'options': {'a': 'Search and mathematical optimization',
   'b': 'Formal logic',
   'c': 'Deep learning',
   'd': 

In [26]:
quiz_sample=json.loads(quiz)

In [27]:
quiz_final=[]
for key,value in quiz_sample.items():
    mcq=value["mcq"]
    options=" | ".join(
        [f"{option} : {option_value}" for option,option_value in value["options"].items()]
    )
    correct=value["correct"]
    quiz_final.append({"mcq":mcq,"options":options,"correct":correct})

In [28]:
quiz_final

[{'mcq': 'What is the broadest sense of Artificial Intelligence (AI)?',
  'options': 'a : Intelligence exhibited by humans | b : Intelligence exhibited by machines | c : Intelligence exhibited by animals | d : Intelligence exhibited by plants',
  'correct': 'b'},
 {'mcq': 'Who was the first person to conduct substantial research in machine intelligence?',
  'options': 'a : John McCarthy | b : Marvin Minksy | c : Nathaniel Rochester | d : Alan Turing',
  'correct': 'd'},
 {'mcq': 'What is the field of research in computer science that develops methods and software for machines to perceive their environment?',
  'options': 'a : Robotics | b : Artificial Intelligence | c : Data Science | d : Cybersecurity',
  'correct': 'b'},
 {'mcq': 'Which technique surpassed all previous AI techniques after 2012?',
  'options': 'a : Search and mathematical optimization | b : Formal logic | c : Deep learning | d : Artificial neural networks',
  'correct': 'c'},
 {'mcq': 'What is among the long-term goal

In [29]:
quiz=pd.DataFrame(quiz_final)

In [30]:
quiz

,mcq,options,correct
0,What is the broadest sense of Artificial Intel...,a : Intelligence exhibited by humans | b : Int...,b
1,Who was the first person to conduct substantia...,a : John McCarthy | b : Marvin Minksy | c : Na...,d
2,What is the field of research in computer scie...,a : Robotics | b : Artificial Intelligence | c...,b
3,Which technique surpassed all previous AI tech...,a : Search and mathematical optimization | b :...,c
4,What is among the long-term goals of AI research?,a : Building robots with emotions | b : Achiev...,b


In [31]:
quiz.to_csv("Artificial Intelligence.csv",index=False)

In [32]:
from datetime import datetime
f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

'07_07_2024_14_34_02.log'